In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.5 MB/s eta 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import json
import time
import datetime
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup

In [ ]:
import torch
torch.manual_seed(64)
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
!pip install -q kaggle

In [ ]:
# !mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d husnifadhilah/poem-generative-indonesia

Dataset URL: https://www.kaggle.com/datasets/husnifadhilah/poem-generative-indonesia
License(s): unknown
 99% 437M/442M [00:10<00:00, 54.3MB/s]
100% 442M/442M [00:10<00:00, 44.1MB/s]


In [ ]:
import zipfile

with zipfile.ZipFile('/content/poem-generative-indonesia.zip', "r") as zip_ref:
    zip_ref.extractall("dataset")

In [ ]:
ls

added_tokens.json  generated_poems.txt     merges.txt         special_tokens_map.json  vocab.json
config.json        generation_config.json  model.safetensors  tokenizer_config.json


In [ ]:
# Loading saved model
model_dir = "/content/dataset"

model = GPT2LMHeadModel.from_pretrained(model_dir)
tokenizer = GPT2Tokenizer.from_pretrained(model_dir)

# model = GPT2LMHeadModel.from_pretrained("prajwalcr/poetry-joy_gpt2")
# tokenizer = GPT2Tokenizer.from_pretrained("prajwalcr/poetry-joy_gpt2")
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50259, bias=False)
)

In [ ]:
import gradio as gr# Function to generate text based on user input


# Function to generate text based on user input
def generate_response(input_text):
    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    # Generate text based on input
    sample_outputs = model.generate(
                                input_ids,
                                do_sample=True,
                                top_k=50,
                                max_length=300,
                                top_p=0.95,
                                num_return_sequences=1
                                )
    # Decode and return the generated text
    generated_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    generated_text = generated_text.replace("\\n ", "\n").replace('\\n',"\n")
    return generated_text.strip()

# Create Gradio interface
input_text = gr.Textbox(lines=7, label="Input Text", placeholder="Enter your message here...")
output_text = gr.Textbox(label="Output Text")

gr.Interface(fn=generate_response, inputs=input_text,
             outputs=output_text, title="Pantun Generator",
             description="Indonesian Pantun Generator GPT-2 chatbot.").launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ee5b458233f747be29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
